In [7]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Generator model
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(28 * 28, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Generator input
latent_dim = 100
generator = build_generator(latent_dim)

# Discriminator input
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))
discriminator.trainable = False

# Combined model
gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

# Training loop
batch_size = 32
epochs = 10000
sample_interval = 1000

(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 127.5 - 1.0
X_train = np.expand_dims(X_train, axis=3)

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Create TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(X_train.shape[0]).batch(batch_size)

@tf.function
def train_step(images):
    noise = tf.random.normal((batch_size, latent_dim))
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)
        disc_loss_real = tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output)

        disc_loss_fake = tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
        disc_loss = 0.5 * (disc_loss_real + disc_loss_fake)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def generate_and_save_images(epoch):
    noise = tf.random.normal((25, latent_dim))
    generated_images = generator(noise, training=False)
    generated_images = 0.5 * generated_images + 0.5

    fig, axs = plt.subplots(5, 5)
    idx = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(generated_images[idx, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            idx += 1
    plt.savefig(f"gan_samples_epoch{epoch}.png")
    plt.close()

# Create optimizers
generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

# Training loop
for epoch in range(epochs):
    for images in dataset:
        train_step(images)

    if epoch % sample_interval == 0:
        print(f"Epoch: {epoch} | Discriminator loss: {disc_loss.numpy()} | Generator loss: {gen_loss.numpy()}")
        generate_and_save_images(epoch)


NameError: name 'disc_loss' is not defined

In [8]:
from PIL import Image

# Open the dog and pizza images
dog_image = Image.open("dog.jpg")
pizza_image = Image.open("pizza.jpg")

# Resize the images to have the same dimensions
dog_image = dog_image.resize(pizza_image.size)

# Create a new image with the same size as the pizza image
composite_image = Image.new("RGB", pizza_image.size)

# Paste the dog image on top of the pizza image
composite_image.paste(pizza_image, (0, 0))
composite_image.paste(dog_image, (0, 0), dog_image)

# Save the composite image
composite_image.save("dog_eating_pizza.jpg")


ValueError: bad transparency mask